In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.style as ms
import librosa
import librosa.display
import IPython.display
import os
import cv2
from os import listdir

from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Flatten, Dropout, Dense, BatchNormalization
from keras import backend as K
from keras.models import load_model
from keras.models import Model
from keras.utils import img_to_array, load_img
from keras.utils import to_categorical
import numpy as np
import tensorflow as tf


from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc 

import matplotlib
import gc
matplotlib.use('Agg') # No pictures displayed 
import pandas as pd

In [2]:
train_path = r"C:\Users\rudra\OneDrive\Desktop\animals\5_Class"
save_path = r"C:\Users\rudra\OneDrive\Desktop\animals\5_Class"
classes = listdir(train_path)

In [3]:
image_list, label_list = [] , []

In [4]:
try:
        print("[INFO] Loading images ...")
        image_folder_list = listdir(train_path)
        for image_folder in image_folder_list:
            print(f"[INFO] Processing {image_folder} ...")
            images_list = listdir(f"{train_path}/{image_folder}/")
            for samples in images_list[:]:
                image_directory = f"{train_path}/{image_folder}/{samples}"
                if image_directory.endswith(".jpg")==True or image_directory.endswith(".JPG")==True or image_directory.endswith(".jpeg")==True:  
                    image_list.append(img_to_array(load_img(image_directory,target_size=(128,128))))
                    label_list.append(image_folder)
    
        print("[INFO] image extract completed")  
        print("total train samples: ")
        print(len(image_list))
except Exception as e:
    print(f"Error : {e}")

[INFO] Loading images ...
[INFO] Processing Aerial ...
[INFO] Processing Aquatic ...
[INFO] Processing Arboreal ...
[INFO] Processing Burrowing ...
[INFO] Processing Terrestrial ...
[INFO] image extract completed
total train samples: 
5399


In [5]:
# # Convert features (X) and labels (y) to Numpy arrays
X = np.array(image_list)/255
y = np.array(label_list)
image_list, label_list = [] , []

In [6]:
le = LabelEncoder()
Y = to_categorical(le.fit_transform(y))

In [7]:
n_classes = len(le.classes_)
print("Total number of classes: ", n_classes)

Total number of classes:  5


In [8]:
labels = classes
print(labels)

['Aerial', 'Aquatic', 'Arboreal', 'Burrowing', 'Terrestrial']


In [9]:
def create_model():
    num_rows = 128
    num_columns = 128
    num_channels = 3
    inputShape = (num_rows, num_columns, num_channels)
    chanDim = -1
    LR = 1e-3
    BATCH_SIZE = 32
    if K.image_data_format() == "channels_first":
        inputShape = (num_rows, num_columns, num_channels)
        chanDim = 1

    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=inputShape))
    model.add(BatchNormalization(axis=chanDim))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(BatchNormalization(axis=chanDim))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
    model.add(BatchNormalization(axis=chanDim))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(len(classes), activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam',metrics=["accuracy"])
    
    return model

In [10]:
from sklearn.metrics import classification_report, precision_recall_fscore_support, confusion_matrix

def compute_confusion_matrix(y_true, y_pred,normalize=False):
    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    return cm

In [11]:
def plot_confusion_matrix(cm,
                          classes, 
                          normalized=False, 
                          title=None, 
                          cmap=plt.cm.Blues,
                          size=(10,10)):
    fig, ax = plt.subplots(figsize=size)
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)

    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalized else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")

    fig.tight_layout()
    plt.show()

In [12]:
def visualize_conv_layers(model, img):
    layer_outputs = [layer.output for layer in model.layers if isinstance(layer, Conv2D)]
    activation_model = Model(inputs=model.input, outputs=layer_outputs)
    activations = activation_model.predict(np.expand_dims(img, axis=0))
    layer = 1
    for layer_activation in activations:
        n_features = layer_activation.shape[-1]
        size = layer_activation.shape[1]
        n_cols = n_features // 16
        display_grid = np.zeros((size * n_cols, 16 * size))
        for col in range(n_cols):
            for row in range(16):
                channel_image = layer_activation[0, :, :, col * 16 + row]
                channel_image -= channel_image.mean()
                channel_image /= channel_image.std()
                channel_image *= 64
                channel_image += 128
                channel_image = np.clip(channel_image, 0, 255).astype('uint8')
                display_grid[col * size : (col + 1) * size, row * size : (row + 1) * size] = channel_image
        scale = 1. / size
        plt.figure(figsize=(scale * display_grid.shape[1], scale * display_grid.shape[0]))
        plt.title(f'Conv Layer {layer}')
        plt.grid(False)
        plt.imshow(display_grid, aspect='auto', cmap='viridis')
        layer = layer + 1 

In [13]:
cv = KFold(n_splits = 3, shuffle=True, random_state=42)
fold_no = 1
acc_per_fold = []
loss_per_fold = []
EPOCHS =20
BATCH_SIZE = 32
gc.collect()
for train,test in cv.split(X,Y):
    model = create_model()
    print(f"[INFO] Training network fold {fold_no}...")
    history = model.fit(X[train],Y[train], batch_size=BATCH_SIZE,
                                  steps_per_epoch=len(X[train]) // BATCH_SIZE,
                                  epochs=EPOCHS, 
                                  verbose=1)
    gc.collect()
    print(f"[INFO] Saving Model fold {fold_no} ...")
    model.save(f"{save_path}/model_5_class_fold"+str(fold_no)+".h5")
    scores = model.evaluate(X[test],Y[test],verbose=0)
    acc_per_fold.append(scores[1]*100)
    loss_per_fold.append(scores[0])
    y_probs = model.predict(X[test])
    # Get predicted labels
    # yhat_probs = list(map(lambda x: 0 if (x<0.5) else 1,y_probs))
    yhat_probs = np.argmax(y_probs, axis=1)
    y_trues = y_trues = np.argmax(Y[test], axis=1)
    cm = confusion_matrix(y_trues, yhat_probs)
    plt.axis('off') # no axis
    # pylab.axes([0., 0., 1., 1.], frameon=False, xticks=[], yticks=[]) # Remove the white edge
    plot_confusion_matrix(cm,
                              labels, 
                              normalized=False, 
                              title=f"Model Performance of 5_class fold: {fold_no}", 
                              cmap=plt.cm.Blues,
                              size=(12,12))
    plt.savefig(f'{save_path}/Model Performance of 5_class fold {fold_no}.jpg', bbox_inches=None, pad_inches=0)
    matplotlib.pyplot.close()
    gc.collect()
    fold_no = fold_no+1

[INFO] Training network fold 1...
Epoch 1/20
112/112 [==============================] - 15s 52ms/step - loss: 1.5073 - accuracy: 0.3758
Epoch 2/20
112/112 [==============================] - 6s 49ms/step - loss: 1.3829 - accuracy: 0.4396
Epoch 3/20
112/112 [==============================] - 5s 49ms/step - loss: 1.3698 - accuracy: 0.4320
Epoch 4/20
112/112 [==============================] - 5s 49ms/step - loss: 1.2247 - accuracy: 0.5102
Epoch 5/20
112/112 [==============================] - 6s 49ms/step - loss: 1.1473 - accuracy: 0.5467
Epoch 6/20
112/112 [==============================] - 6s 49ms/step - loss: 1.0492 - accuracy: 0.5879
Epoch 7/20
112/112 [==============================] - 6s 50ms/step - loss: 0.8668 - accuracy: 0.6751
Epoch 8/20
112/112 [==============================] - 6s 49ms/step - loss: 0.7062 - accuracy: 0.7438
Epoch 9/20
112/112 [==============================] - 5s 49ms/step - loss: 0.5795 - accuracy: 0.7939
Epoch 10/20
112/112 [==============================] - 5

C:\Users\rudra\AppData\Local\Temp\ipykernel_24632\3249705492.py:34: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


[INFO] Training network fold 2...
Epoch 1/20
112/112 [==============================] - 7s 50ms/step - loss: 1.5192 - accuracy: 0.3691
Epoch 2/20
112/112 [==============================] - 6s 50ms/step - loss: 1.3344 - accuracy: 0.4623
Epoch 3/20
112/112 [==============================] - 5s 49ms/step - loss: 1.2087 - accuracy: 0.5111
Epoch 4/20
112/112 [==============================] - 6s 49ms/step - loss: 1.0506 - accuracy: 0.5868
Epoch 5/20
112/112 [==============================] - 5s 49ms/step - loss: 0.8809 - accuracy: 0.6678
Epoch 6/20
112/112 [==============================] - 5s 49ms/step - loss: 0.6971 - accuracy: 0.7365
Epoch 7/20
112/112 [==============================] - 6s 49ms/step - loss: 0.5143 - accuracy: 0.8175
Epoch 8/20
112/112 [==============================] - 5s 49ms/step - loss: 0.3731 - accuracy: 0.8738
Epoch 9/20
112/112 [==============================] - 5s 49ms/step - loss: 0.2778 - accuracy: 0.9111
Epoch 10/20
112/112 [==============================] - 6s

C:\Users\rudra\AppData\Local\Temp\ipykernel_24632\3249705492.py:34: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


[INFO] Training network fold 3...
Epoch 1/20
112/112 [==============================] - 7s 50ms/step - loss: 1.4562 - accuracy: 0.3951
Epoch 2/20
112/112 [==============================] - 6s 51ms/step - loss: 1.2847 - accuracy: 0.4773
Epoch 3/20
112/112 [==============================] - 6s 52ms/step - loss: 1.1935 - accuracy: 0.5305
Epoch 4/20
112/112 [==============================] - 6s 50ms/step - loss: 1.3114 - accuracy: 0.4664
Epoch 5/20
112/112 [==============================] - 6s 50ms/step - loss: 1.1030 - accuracy: 0.5605
Epoch 6/20
112/112 [==============================] - 6s 50ms/step - loss: 0.9642 - accuracy: 0.6326
Epoch 7/20
112/112 [==============================] - 6s 49ms/step - loss: 0.7617 - accuracy: 0.7195
Epoch 8/20
112/112 [==============================] - 5s 49ms/step - loss: 0.5917 - accuracy: 0.7923
Epoch 9/20
112/112 [==============================] - 6s 49ms/step - loss: 0.3798 - accuracy: 0.8730
Epoch 10/20
112/112 [==============================] - 6s

C:\Users\rudra\AppData\Local\Temp\ipykernel_24632\3249705492.py:34: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [14]:
acc_per_fold

[58.83333086967468, 56.166666746139526, 59.75542068481445]

In [15]:
loss_per_fold

[1.9911105632781982, 1.8193013668060303, 2.0265347957611084]

In [16]:
visualize_conv_layers(model,img_to_array(load_img(r"C:\Users\rudra\OneDrive\Desktop\animals\OvR3\antelope\rest\0a1f4371e3.jpg",target_size=(128,128)))/255)

1/1 [==============================] - 0s 144ms/step


C:\Users\rudra\AppData\Local\Temp\ipykernel_24632\1767733992.py:15: RuntimeWarning: invalid value encountered in divide
  channel_image /= channel_image.std()
C:\Users\rudra\AppData\Local\Temp\ipykernel_24632\1767733992.py:18: RuntimeWarning: invalid value encountered in cast
  channel_image = np.clip(channel_image, 0, 255).astype('uint8')


In [17]:
plt.savefig(f'{save_path}/Conv Visualization of 5_class.jpg', bbox_inches=None, pad_inches=0)